## Imports

In [8]:
import pandas as pd

## Load Data

In [9]:
train_df = pd.read_csv('../data/processed/subtask1_train_bert.csv')

In [10]:
train_df.head()

,sentenceID,label,text
0,100000,1,goodfellows theory has been questioned however...
1,100001,1,however both campaigners and propeoples vote m...
2,100002,1,things could have been even better if the whol...
3,100003,0,the new request if approved would keep the mil...
4,100004,0,companies in financial difficulty can currentl...


## Basic Check

In [ ]:
train_df[train_df['text'].str.contains('with')].iloc[3]['text']

In [ ]:
train_df[train_df['text'].str.contains('with')].groupby('label').count()

## Splitting the dataset
#### Wish - 0, Should - 1, Could/Would - 2, If - 3, Other - 4

In [11]:
from nltk.tokenize import word_tokenize

In [12]:
id_to_word = {0: 'wish',
             1: 'should',
             2: 'couldwould',
             3: 'if',
             4: 'other'}

In [13]:
def check(ll, idx):
    if idx == 0:
        return 'wish' in ll
    elif idx == 1:
        return 'should' in ll
    elif idx == 2:
        return ('could' in ll) or ('would' in ll)
    elif idx == 3:
        return 'if' in ll
    elif idx == 4:
        return ('wish' not in ll) and ('should' not in ll) and ('could' not in ll) and ('would' not in ll) and ('if' not in ll)

In [14]:
for id_num in range(5):
    idxs = [i for i, x in enumerate(list(train_df['text'])) if check(word_tokenize(x), id_num)]
    new_train_df = train_df.iloc[idxs]
    print(new_train_df.groupby('label').count())
#     new_train_df.to_csv('../data/processed/subtask1_train_bert_'+id_to_word[id_num]+'.csv', index=False)

       sentenceID  text
label                  
0             280   280
1             119   119
       sentenceID  text
label                  
0             778   778
1             118   118
       sentenceID  text
label                  
0            5297  5297
1             996   996
       sentenceID  text
label                  
0            8498  8498
1             738   738
       sentenceID  text
label                  
0            1482  1482
1             114   114


## Emsemble Testing

In [28]:
test_df = pd.read_csv('../data/processed/subtask1_test_bert.csv')

In [29]:
import numpy as np
with open('../data/processed/if.npy', 'rb') as f:
    if_arr = np.load(f)
with open('../data/processed/should.npy', 'rb') as f:
    should_arr = np.load(f)
with open('../data/processed/couldwould.npy', 'rb') as f:
    cw_arr = np.load(f)
with open('../data/processed/wish.npy', 'rb') as f:
    wish_arr = np.load(f)
with open('../data/processed/other.npy', 'rb') as f:
    other_arr = np.load(f)

In [30]:
from scipy import stats

In [52]:
fin_ans = []
for i, t in enumerate(list(test_df['text'])):
    ll = word_tokenize(t)
    ans = []
    if 'wish' in ll:
        ans.append(wish_arr[i])
    if 'should' in ll:
        ans.append(should_arr[i])
    if ('could' in ll) or ('would' in ll):
        ans.append(cw_arr[i])
    if 'if' in ll:
        ans.append(if_arr[i])
    if ('wish' not in ll) and ('should' not in ll) and ('could' not in ll) and ('would' not in ll) and ('if' not in ll):
        ans.append(other_arr[i])
    if sum(ans) > 0:
        fin_ans.append(1)
    else:
        fin_ans.append(0)

In [53]:
pred_labels = np.array(fin_ans)
gold_labels = np.array(test_df['label'])

In [54]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

print('F1: ', f1_score(gold_labels, pred_labels))
print('Accuracy: ', accuracy_score(gold_labels, pred_labels))
print('Precision: ', precision_score(gold_labels, pred_labels))
print('Recall: ', recall_score(gold_labels, pred_labels))

F1:  0.8619756427604871
Accuracy:  0.9708571428571429
Precision:  0.8608108108108108
Recall:  0.8631436314363143
